In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import my_lib
import csv
import tensorflow as tf

... OK (Got response in 0.49 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [5]:
pipeline = my_lib.Pipeline()
raw_df = pipeline.load_data()
raw_df = pipeline.encode_to_category(raw_df)
raw_df = pipeline.tokenize(raw_df)
raw_df.head()




,tweet,label,enterprise,is_irrelevant,token_list,clean_tweet,number_vector,input_vector
0,20 min line @apple store @short pump.,1,0,0,20 min line @apple store @short pump.,20 min line @apple store @short pump .,"[68, 185, 431, 268, 128, 606, 431, 329, 128, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Nueva tecnología convierte cualquier superfici...,3,1,1,nueva tecnología convierte cualquier superfici...,nueva tecnología convierte cualquier superfici...,"[606, 194, 638, 661, 415, 431, 652, 638, 472, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,Some people should not post replies in #Google...,1,2,0,some people should not post replies in #google...,some people should not post replies in #google...,"[15, 302, 268, 638, 431, 57, 638, 302, 57, 329...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,I know a few others having same issue RT @Joel...,2,0,0,i know a few others having same issue rt @joel...,i know a few others having same issue rt @joel...,"[128, 431, 317, 606, 302, 650, 431, 415, 431, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,"#Microsoft - We put the """"backwards"""" into bac...",2,1,0,"#microsoft - we put the """"backwards"""" into bac...","#microsoft - we put the "" "" backwards "" "" ...","[220, 268, 128, 472, 86, 302, 15, 302, 331, 65...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
# df = raw_df.loc[raw_df.label.isin([0,2])]
df = raw_df.loc[raw_df.label.isin([0,1,2,3])]
df = my_lib.processing.remove_useless_col(df, keep_col_list=["label", "clean_tweet"])
df_train_data, df_test_data = my_lib.processing.split_data(df)
print(f"SHAPE -> train : {df_train_data.shape}, test : {df_test_data.shape}")
df.head()

SHAPE -> train : (3338, 2), test : (835, 2)


,label,clean_tweet
0,1,20 min line @apple store @short pump .
1,3,nueva tecnología convierte cualquier superfici...
2,1,some people should not post replies in #google...
3,2,i know a few others having same issue rt @joel...
4,2,"#microsoft - we put the "" "" backwards "" "" ..."


In [7]:
for label_id, label_name in enumerate(["pos", "neu", "neg", "irr"]):
    tmp_df = df.loc[df["label"]==label_id, ["clean_tweet"]]
    for idx, row in tmp_df.iterrows():
        row.to_csv(f"/data/tensorflow_bert_data/{label_name}/text_{idx}.txt", sep=" ", index=None,
                quotechar=" ", header=False)

In [8]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import numpy as np

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", num_labels=4)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
data = tf.keras.preprocessing.text_dataset_from_directory('/data/tensorflow_bert_data/', seed=123, batch_size=30000)
data

Found 4173 files belonging to 4 classes.


<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>

In [10]:
for i in data.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

df_data = pd.DataFrame([train_feat, train_lab]).T
df_data.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df_data['DATA_COLUMN'] = df_data['DATA_COLUMN'].str.decode("utf-8")
print(df_data.shape)
df_data.head()

(4173, 2)


,DATA_COLUMN,LABEL_COLUMN
0,rt @remcovandenhout : meest comfortabel...,0
1,omg @apple why the fuck did you delete...,1
2,rt @lcmediahouse : 4chan ' s chris p...,1
3,#twitter keeps me from sleeping ! #so...,2
4,would it kill @apple to put a braille ...,1


In [11]:
row_number = len(df_data)
print(row_number)
df_train = pd.DataFrame()
df_test = pd.DataFrame()

for i in range(3):
    tmp_df = df_data.loc[df_data["LABEL_COLUMN"]==i]

    msk = np.random.rand(len(tmp_df)) < 0.8
    df_train = pd.concat([df_train, tmp_df[msk]])
    df_test = pd.concat([df_test, tmp_df[~msk]])

df_train.shape, df_test.shape

4173


((2970, 2), (776, 2))

In [12]:
df_train[["DATA_COLUMN", "LABEL_COLUMN"]].groupby("LABEL_COLUMN").count()

,DATA_COLUMN
LABEL_COLUMN,
0,1099
1,391
2,1480


In [13]:
df_test[["DATA_COLUMN", "LABEL_COLUMN"]].groupby("LABEL_COLUMN").count()

,DATA_COLUMN
LABEL_COLUMN,
0,290
1,95
2,391


In [14]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_test = shuffle(df_test)

In [15]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(df_train,
                                                                           df_test,
                                                                           'DATA_COLUMN',
                                                                           'LABEL_COLUMN')

In [16]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [17]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(df_train, df_test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [18]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)


Epoch 1/2
      7/Unknown - 76s 76s/step - loss: 1.4721 - accuracy: 0.1250 - 93s 17s/step - loss: 1.4610 - accuracy: 0.1484 - 110s 17s/step - loss: 1.4297 - accuracy: 0.1997 - 124s 16s/step - loss: 1.4081 - accuracy: 0.2240 - 139s 16s/step - loss: 1.3911 - accuracy: 0.2429 - 155s 16s/step - loss: 1.3733 - accuracy: 0.2641 - 170s 16s/step - loss: 1.3557 - accuracy: 0.2850

In [ ]:
# pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
#                   'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']
#
# tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
# tf_outputs = model(tf_batch)
# tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
# labels = ['Negative','Positive']
# label = tf.argmax(tf_predictions, axis=1)
# label = label.numpy()
# for i in range(len(pred_sentences)):
#   print(pred_sentences[i], ": \n", labels[label[i]])

In [ ]:
model.save('/app/models/multi_class_bert_2')




In [ ]:
print("a")


In [ ]:
labels = ['irr','neg', 'neu', 'pos']

def predict(sentence_list):

    tf_batch = tokenizer(sentence_list, max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    return label
    # for i in range(len(sentence_list)):
    #   print(sentence_list[i], ": \n", labels[label[i]])


In [ ]:

sen_list = list(df_test["DATA_COLUMN"].values)
res_pred = predict(sen_list)

In [ ]:
res_pred

In [ ]:
df_re = df_test.copy()
df_re["predict"] = res_pred
df_re.shape

In [ ]:
df_re.loc[df_re["LABEL_COLUMN"] != df_re["predict"]].head(60)

In [ ]:

m = my_lib.model.BERT().load("/app/models/multi_class_bert")

In [ ]:
m.predict(["Hi how are you my friend"])


In [ ]:

tf_batch = m.tokenizer(["Hi how are you my friend."], max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = m.model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
label